<a href="https://colab.research.google.com/github/guillermohenrion/Social-Network-Analytics/blob/master/nps_tweet_ant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from twython import Twython
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#from PIL import Image
from os import path
import matplotlib.pyplot as plt
import os
import random
import pandas as pd
import ftfy
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import gensim
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
import os
from gensim.models.wrappers import LdaMallet, ldamallet
from gensim.corpora import Dictionary
import spacy
import pyLDAvis.gensim
import csv
from unicodedata import normalize
import string

SEED = 8888
#from wordcloud import WordCloud, STOPWORDS


twitter = Twython("U63VxPMqLMwTPFUWEcPUeWMng", "sbz3lDj50wMdteqmDCXIexM7wuCxapGs89GtOlSDZV5C1W1hQz",
                    "280807268-MgZfUIUEb1vkPehfjB8sfuicMstiw0Afek49kkNL", "g8QammrsFv1zHfWrGtypSkNloEx1tmU2IpzRaW6JJNjIX")
twitter.sleep_on_rate_limit = True

In [ ]:
def sacarURL(t):
    p=re.compile('<a [^>]*>')
    p2=re.compile('</a>')
    r=re.sub(p, '', t)
    r=re.sub(p, '', t)
    r=re.sub(p, '', t)
    r=re.sub(p2, '', r)
    r=re.sub(p2, '', r)
    r=re.sub(p2, '', r)
    return r


In [ ]:
######################################################################################################################
# Funciones
######################################################################################################################

def cleaner(text, is_topic=False):
    if not is_topic:
        text = text.lower()  # texto a minúsculas
        text = re.sub(r'\[.*?¿\]%', ' ', text)  # Se remueven corchetes
        text = re.sub(r'\(.*?¿\)%', ' ', text)  # Se remueven parentesis
        text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)  # Se remueven signos de puntuación
        text = re.sub(r'\w*\d\w*', '', text)  # Se remueven palabras que contienen dígitos.
        text = re.sub(r'\n', ' ', text)  # Sustituye nueva línea por un espacio
        text = re.sub(r'\s+', ' ', text, flags=re.I)  # Sustituye los múltiples espacios por sólo un espacio
        text = re.sub(r'[‘’“”…«»°*_/¿¡!?]', '', text)  # Remueve otros caracteres no sintácticos
        text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+|\u20AC", r"\1",
                      normalize("NFD", text), 0, re.I)  # NFD y eliminar diacríticos y símbolo euro
        text = normalize('NFC', text)  # NFC
        text_list = []
        for w1 in text.split(' '):
            text_list.append(w1)
        text_non_stop = [word for word in text_list if word not in stopwords.words()]
        return text_non_stop
    else:
        text_non_stop = [word for word in text if word not in stopwords.words()]
        bigram_list = bigram[text_non_stop]
        out_text = lemmatization(' '.join(bigram_list), allowed_postags=['NOUN'])  # , 'ADJ', 'PROPN'
        return out_text


def lemmatization(texts, allowed_postags=None):
    if allowed_postags is None:
        allowed_postags = ['NOUN']
    texts_out = [token.lemma_ for token in nlp(texts) if token.pos_ in
                 allowed_postags and token.text not in black_list and len(token.text) > 2]
    return texts_out


def display_topics(model, model_type='lda'):
    for topic_idx, topic in enumerate(model.print_topics()):
        print('Topic %d:' % topic_idx)
        if model_type == 'hdp':
            print(" ".join(re.findall(r'\*(.[^\*-S]+).?', topic[1])), '\n')
        else:
            print(" ".join(re.findall(r'\"(.[^"]+).?', topic[1])), '\n')


def evaluate_graph(dictionary, corpus, texts, limit, model):
    """
    Function to display num_topics - LDA graph using c_v coherence

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit

    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    for num_topics in range(1, limit):
        if model == 'lsi':
            lm = LsiModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        elif model == 'mallet':
            lm = LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
        else:
            lm = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v.append(cm.get_coherence())

    # Show graph
    x = range(1, limit)
    plt.plot(x, c_v)
    plt.xlabel('num_topics')
    plt.ylabel('Coherence score')
    plt.legend('c_v', loc='best')
    plt.show()

    return lm_list, c_v


def format_topics_sentences(ldamodel=0, corpus=[], texts=0):
    sent_topics_df = pd.DataFrame()  # - n

    # Tema principal en cada documento
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Tema dominante, porcentaje de contribución y palabras claves
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # Tema dominante
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(
                    pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Agrega el texto original al final
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return sent_topics_df


######################################################################################################################
# Carga de archivos
######################################################################################################################

In [ ]:
black_list = ['n', 'q', 'eh', 'me', 'xq', 'z', 'x', 'etc', 'd', 'm', 's', 'u', 'p', 'l', 'as', 'av', 'j', 'porq', 'c',
              'v', 'cdo', 'i', 'unas', 'k', 't', 'pq', 'b', 'g', 'ne', 'r', 'h', 'qu', 'f', 'co', 'ud', 'sta', 'srta',
              'sr', 'sra', 'srita', 'tb', 'osea', 'po', 'ala', 'ami', 'ka', 'mo', 'migo', 'lis', 'in', 'idas', 'asique',
              'aunq', 'miy', 'lla', 'unlam', 'na', 'don', 'xxxx', 'yla', 'qie', 'qur', 'qye', 'fu', 'my', 'ke', 'laa',
              'll', 'mm', 'tmb', 'pr', 'ay', 'or', 'nc', 'xk', 'xx', 'ye', 'rn', 'hr', 'eramos', 'estabamos', 'estais',
              'estan', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos',
              'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'fueramos',
              'fuesemos', 'habeis', 'habia', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras',
              'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos',
              'hubiesemos', 'mas', 'mia', 'mias', 'mio', 'mios', 'seais', 'sera', 'seran', 'seras', 'sere', 'sereis',
              'seria', 'seriais', 'seriamos', 'serian', 'serias', 'si', 'tambien', 'tendra', 'tendran', 'tendras',
              'tendre', 'tendreis', 'tendria', 'tendriais', 'tendriamos', 'tendrian', 'tendrias', 'teneis', 'tengais',
              'tenia', 'teniais', 'teniamos', 'tenian', 'tenias', 'tuvieramos', 'tuviesemos', 'vez']

In [ ]:
#stop = set(stopwords.words('spanish'))
#additional_stopwords = set(black_list)
#stopwords = stop.union(additional_stopwords)

nlp = spacy.load('es_core_news_sm')


In [ ]:
user_tweets = twitter.get_user_timeline(screen_name='alferdez',
                                        include_rts=True, count=200, tweet_mode="extended")
tweetdf=pd.DataFrame([], columns=['id', 'text'])
for tweet in user_tweets:
    tweet['text'] = Twython.html_for_tweet(tweet)
    new_row={'id':tweet['id'], 'text':sacarURL(tweet['text'])}
    tweetdf=tweetdf.append(new_row, ignore_index=True)

In [ ]:
tweetdf['text_orig'] = tweetdf['text']  # Resguardamos original
tweetdf['text'] = tweetdf['text'].apply(cleaner, args=(False,))


In [ ]:
bigram = gensim.models.Phrases(tweetdf.text.to_list(), min_count=5, threshold=10.0)
tweetdf['text'] = tweetdf['text'].apply(cleaner, args=(True,))

In [ ]:
tweetdf

,id,text,text_orig
0,1273464435684900864,"[medir, cantidad, gente, lugar, motivar, creci...",La medida apunta a reducir la cantidad de gent...
1,1273464433440952325,"[crecimiento, contagio, partir, viernes, trans...",Tras una reunión con @Kicillofok y @diegosanti...
2,1273462599313756160,"[anunciar, transportar, trabajador, medir, aut...",IMPORTANTE‼️ @alferdez anunció que desde maña...
3,1273460113114791936,"[problema, diferenciar, suffix_twitter]",Esta noche el Presidente @alferdez nos convocó...
4,1273459546783129605,"[salud, coser, vida, muerto, suffix_twitter]","""No tengo el dilema entre la salud y la econom..."
...,...,...,...
195,1263828658017439745,"[moldavskyrober, abrazar, filmina]","Jajaja me has hecho reír mucho, querido @molda..."
196,1263777085627252739,"[empezo, pyme, hecho, empresa, programar, empl...",Ni bien empezó la pandemia el presidente @alfe...
197,1263637108046127104,"[compromiso, encerrar, suffix_twitter]",No vamos a asumir ningún compromiso con nuestr...
198,1263624086179151872,"[provincia, actividad, protocolo, severo, resp...","También visité Tucumán, una provincia que se m..."


In [ ]:
# Corpus y diccionario para gensim
dictionary = Dictionary(tweetdf['text'].to_list())
dictionary.compactify()
# Filtrado de extremos
dictionary.filter_extremes(no_below=2, no_above=0.97, keep_n=None)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in tweetdf['text'].to_list()]

In [ ]:
lmlist_lsi, c_v = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=tweetdf['text'].to_list(),
                                 limit=50, model='lsi')  # 10, 30?


ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary)
display_topics(ldamodel)

In [ ]:
lsimodel = LsiModel(corpus=corpus, num_topics=5, id2word=dictionary)
display_topics(lsimodel)

In [ ]:
print(lmlist_lsi)
print(c_v)